Example robots come from example-robot-data package

https://github.com/Gepetto/example-robot-data

In [2]:
import example_robot_data
import pinocchio as pin

One Option is to manually point to the directory. When you install with robot_pkg, it seems to install in /opt/openrobots/share/

The urdf_path should point exactly to the `.urdf` file. The mesh_dir should point to the folder such that the `<mesh filename="package://example-robot-data/robots...>` can resolve

```xml
<visual>
    <origin rpy="0 0 0" xyz="0 0 0"/>
    <geometry>
    <mesh filename="package://example-robot-data/robots/solo_description/meshes/stl/solo12/solo_12_base.stl"/>
    </geometry>
    <material name="grey">
    <color rgba="0.8 0.8 0.8 1.0"/>
    </material>
</visual>
```

In [ ]:
mesh_dir = "/opt/openrobots/share/" # should be parent directory for mesh file name
urdf_model_path = "/opt/openrobots/share/example-robot-data/robots/solo_description/robots/solo12.urdf" 

model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_model_path, mesh_dir, pin.JointModelFreeFlyer())
print(model)

Nb joints = 14 (nq=19,nv=18)
  Joint 0 universe: parent=0
  Joint 1 root_joint: parent=0
  Joint 2 FL_HAA: parent=1
  Joint 3 FL_HFE: parent=2
  Joint 4 FL_KFE: parent=3
  Joint 5 FR_HAA: parent=1
  Joint 6 FR_HFE: parent=5
  Joint 7 FR_KFE: parent=6
  Joint 8 HL_HAA: parent=1
  Joint 9 HL_HFE: parent=8
  Joint 10 HL_KFE: parent=9
  Joint 11 HR_HAA: parent=1
  Joint 12 HR_HFE: parent=11
  Joint 13 HR_KFE: parent=12



In [20]:
# The package is also aware of the install paths
from example_robot_data.robots_loader import EXAMPLE_ROBOT_DATA_SOURCE_DIR, EXAMPLE_ROBOT_DATA_MODEL_DIR
print(EXAMPLE_ROBOT_DATA_SOURCE_DIR)
print(EXAMPLE_ROBOT_DATA_MODEL_DIR)


/local/robotpkg/var/tmp/robotpkg/robots/py-example-robot-data/work/example-robot-data-4.2.0/robots
/opt/openrobots/share/example-robot-data/robots


In [35]:
# So you can directly load with helpers
robot = example_robot_data.load('ur5')
model = robot.model
data = model.createData()

print(model)
print(data)








Nb joints = 7 (nq=6,nv=6)
  Joint 0 universe: parent=0
  Joint 1 shoulder_pan_joint: parent=0
  Joint 2 shoulder_lift_joint: parent=1
  Joint 3 elbow_joint: parent=2
  Joint 4 wrist_1_joint: parent=3
  Joint 5 wrist_2_joint: parent=4
  Joint 6 wrist_3_joint: parent=5



In [58]:
for i, name in enumerate(model.names):
    print(f"Joint ID {i}: {name}")

Joint ID 0: universe
Joint ID 1: shoulder_pan_joint
Joint ID 2: shoulder_lift_joint
Joint ID 3: elbow_joint
Joint ID 4: wrist_1_joint
Joint ID 5: wrist_2_joint
Joint ID 6: wrist_3_joint


In [59]:
model.names[6]

'wrist_3_joint'

In [52]:
print(f"[FrameId] frame_name - frame_type - parent_joint")

for i, frame in enumerate(model.frames):
    print(f"[{i}] {frame.name} - {frame.type} - {frame.parentJoint}")


[FrameId] frame_name - frame_type - parent_joint
[0] universe - FIXED_JOINT - 0
[1] world - BODY - 0
[2] world_joint - FIXED_JOINT - 0
[3] base_link - BODY - 0
[4] base_link-base_fixed_joint - FIXED_JOINT - 0
[5] base - BODY - 0
[6] shoulder_pan_joint - JOINT - 1
[7] shoulder_link - BODY - 1
[8] shoulder_lift_joint - JOINT - 2
[9] upper_arm_link - BODY - 2
[10] elbow_joint - JOINT - 3
[11] forearm_link - BODY - 3
[12] wrist_1_joint - JOINT - 4
[13] wrist_1_link - BODY - 4
[14] wrist_2_joint - JOINT - 5
[15] wrist_2_link - BODY - 5
[16] wrist_3_joint - JOINT - 6
[17] wrist_3_link - BODY - 6
[18] ee_fixed_joint - FIXED_JOINT - 6
[19] ee_link - BODY - 6
[20] wrist_3_link-tool0_fixed_joint - FIXED_JOINT - 6
[21] tool0 - BODY - 6


In [48]:
print(model.getJointId("wrist_3_link"))
print(model.getJointId("ee_link"))
print(model.getJointId("tool0"))


7
7
7


In [49]:
print(model.frames[model.getFrameId("tool0")].parentJoint)
print(model.frames[model.getFrameId("wrist_3_link")].parentJoint)
print(model.frames[model.getFrameId("ee_link")].parentJoint)


6
6
6


In [42]:
joint_id = model.getJointId("tool0")
frame_id = model.getFrameId("tool0")
parent_joint_id = model.frames[frame_id].parentJoint

print("joint_id", joint_id)
print("frame_id", frame_id)
print("parent_joint_id", parent_joint_id)

joint_id 7
frame_id 21
parent_joint_id 6


In [46]:
q = pin.neutral(model)
# BUG: this doesn't update the frame placements 
pin.forwardKinematics(model, data, q) # set starting position
T_joint = data.oMi[parent_joint_id]
T_frame = data.oMf[frame_id]

print(T_joint)
print(T_frame)
print("")
pin.framesForwardKinematics(model, data, q)
T_joint = data.oMi[parent_joint_id]
T_frame = data.oMf[frame_id]

print(T_joint)
print(T_frame)


  R =
          -1            0  9.79328e-12
           0            1            0
-9.79328e-12            0           -1
  p =   0.81725   0.10915 -0.005491

  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0


  R =
          -1            0  9.79328e-12
           0            1            0
-9.79328e-12            0           -1
  p =   0.81725   0.10915 -0.005491

  R =
          -1 -9.79328e-12  4.79541e-23
           0  4.89664e-12            1
-9.79328e-12            1 -4.89664e-12
  p =   0.81725   0.19145 -0.005491



One issue is that the frames and joints are different! what to do....

The more general solution is to do with with frames. beacuse then you can solve the IK between any two links... you can use caching as well...

the jacobean can always be calculated via the 6th joint...

so You can pass in the IK tip with what ever, but then I need to transfrom from IK tip to Jacobean I think no?



In [54]:
pin.framesForwardKinematics(model, data, q)
print(data.oMf[model.getFrameId("wrist_3_joint")])
print(data.oMf[model.getFrameId("wrist_3_link")])

  R =
          -1            0  9.79328e-12
           0            1            0
-9.79328e-12            0           -1
  p =   0.81725   0.10915 -0.005491

  R =
          -1            0  9.79328e-12
           0            1            0
-9.79328e-12            0           -1
  p =   0.81725   0.10915 -0.005491

